In [ ]:
## starting model refinement and testing in this notebook 

# Import our dependencies
import pandas as pd
from datetime import datetime
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score

# Read in our ramen data
housing_data_df= pd.read_csv("../Resources/City_Zhvi_AllHomes_for_ML_Linear_Regression.csv",parse_dates=True)
housing_data_df

In [ ]:
#Make a top city dataframe based on size rank of each city
topcity = housing_data_df[['SizeRank', 'RegionName']].copy()
topcity

In [ ]:
#Reduce the top city dataframe to the top 60 cities by size rank
topcity = topcity.loc[0:59,'RegionName']

In [ ]:
#Covert city dataframe to list
topcity.values.tolist()

In [ ]:
## start dropping unneeded columns
## dropped RegionID, State, Metro, CountyName 
housing_data_df=housing_data_df.drop(housing_data_df.iloc[:, 0:3],axis = 1)
housing_data_df.head()

In [ ]:
## drop remaining unneeded columns
## dropped all months before January 2010
housing_data_df=housing_data_df.drop(housing_data_df.iloc[:, 1:174],axis = 1)
housing_data_df.head()

In [ ]:
#Dropping all but first value in RegionName. City names appear more than once if there are cities in diffrent states 
# with the same name. Since we are looking for top cities we are ok to drop all occurances after the first.
housing_data_df.drop_duplicates(subset=['RegionName'], inplace=True)
housing_data_df

In [ ]:
#keep only rows with the cities in our top cities by size
housing_topcity_df = housing_data_df[housing_data_df['RegionName'].isin(topcity)]
housing_topcity_df

In [ ]:
## transpose dates to rows
housing_topcity_df=housing_topcity_df.transpose()
housing_topcity_df.head()

In [ ]:
#move first row to column names
housing_topcity_df.columns = housing_topcity_df.iloc[0]
housing_topcity_df

In [ ]:
#remove first row shwowing city name
housing_topcity_df = housing_topcity_df.iloc[1: , :]
housing_topcity_df

In [ ]:
## check that KNN imputer worked 

    
for column in housing_topcity_df.columns:
    print(f"column {column} has {housing_topcity_df[column].isnull().sum()} null values")

In [ ]:
housing_topcity_df.dropna(axis=1)
len(housing_topcity_df)

In [ ]:
zips= housing_topcity_df.columns.tolist()
len(zips)

In [ ]:
housing_topcity_df.to_csv("housing_topcity.csv", encoding='utf-8')

In [ ]:
# sf but area code is wrong, IN = 47401

# removing last 20 for testing set 0:271
training_set = housing_topcity_df.iloc[:103, 0:1].values
training_set

In [ ]:
len(training_set)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
# Creating a data structure with 60 timesteps and 1 output
import numpy as np
X_train = []
y_train = []
for i in range(60, 103):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
len(X_train)

In [ ]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
# Part 3 - Making the predictions and visualising the results

# Getting the real stock price
dataset_test = pd.read_csv("housing_topcity.csv")
dataset_test.head()

In [ ]:
real_housing_price = dataset_test.iloc[:, 1:2].values
real_housing_price

In [ ]:
training= dataset_test.iloc[0:103, 1:2]
print(len(training))
training

In [ ]:
# make DF to set up input for testing 

training = housing_topcity_df.iloc[:, 0:1]
training.reset_index(drop=True, inplace=True)
training

In [ ]:
test = housing_topcity_df.iloc[103:, 0:1]
test.reset_index(drop=True, inplace=True)
print(len(test))
test

In [ ]:
type(test)

In [ ]:
type(training)

In [ ]:
# Getting the predicted house prices
dataset_total = pd.concat((training, test), axis = 0)
dataset_total

In [ ]:
inputs = dataset_total[len(dataset_total) - len(test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [ ]:
len(inputs)

In [ ]:
X_test = []

for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_housing_price = regressor.predict(X_test)
predicted_housing_price = sc.inverse_transform(predicted_housing_price)

In [ ]:
predicted_housing_price

In [ ]:
forgraph=real_housing_price[103:]
forgraph

In [ ]:
import matplotlib.pyplot as plt
# Visualising the results
plt.plot(forgraph, color = 'red', label = 'Real housing Price')
plt.plot(predicted_housing_price, color = 'blue', label = 'Predicted housing Price')
plt.title('New York Price Predictions')
plt.xlabel('Time')
plt.ylabel('Housing Price')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
RNN_mse = mean_squared_error(forgraph,predicted_housing_price)
RNN_rmse=np.sqrt(RNN_mse)
print("mse :",RNN_mse)
print("rmse :",RNN_rmse)

from sklearn.metrics import mean_absolute_error
RNN_MAE = mean_absolute_error(forgraph,predicted_housing_price)
print("MAE :",RNN_MAE)

